In [24]:
import shioaji as sj

In [25]:
api = sj.Shioaji(simulation=True)
api.login(person_id="PAPIUSER01",
          passwd="2222",
          contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
          )

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
def InitSimulation(userId : int = 1):
    name = "PAPIUSER0" + str(userId)
    pwd = "2222"
    api = sj.Shioaji(simulation=True)    
    api.login(person_id="PAPIUSER01",
          passwd="2222",
          contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
          )

In [5]:
#取得帳號清單
accounts = api.list_accounts()

In [6]:
#取得預設帳戶
stkAcc = api.stock_account
futAcc = api.futopt_account

In [26]:
def SendOrder(symbolId, orderPrice, orderQty, bsType, priceType, timeInForce, sessionType, isFirstSell, accountType):
    contract = __getContract(symbolId, accountType)    
    #使用預設帳號下單
    order = api.Order(
                      price = orderPrice,
                      quantity = orderQty,
                      action = __getAction(bsType),
                      price_type = __getPriceType(priceType),
                      order_type = __getTimeInForce(timeInForce),                      
                      order_lot = __getSessionType(sessionType),
                      first_sell = isFirstSell,
                      account = __getAccountInfo(accountType)
                     )
    trade = api.place_order(contract, order)
    return trade

In [29]:
SendOrder("2886", 10, 1, "Buy", "Limit", "ROD", "RegularTrading", False, "Stk")

AttributeError: 'NoneType' object has no attribute 'Stocks'

In [15]:
#取得買賣類型
def __getAction(bsType):
    if bsType == "Buy":
        return sj.constant.Action.Buy
    elif bsType == "Sell":
        return sj.constant.Action.Sell
    else:
        raise Exception("Not correct Action: " + bsType + ", only can input {Buy: 買, Sell: 賣}")

In [46]:
GetAction("")

Exception: Not correct Action: , only can input {B: 買, S: 賣}

In [16]:
#取得價格類型
def __getPriceType(priceType):
    if priceType == "Limit": #限價
        return sj.constant.StockPriceType.LMT
    elif priceType == "Market": #市價
        return sj.constant.StockPriceType.MKT
    elif priceType == "RangeMarket": #範圍市價
        return sj.constant.StockPriceType.MKP
    else:
        raise Exception("Not correct StockPriceType: " + priceType + 
                        ", only can input {Limit: 限價, Market: 市價, RangeMarket: 範圍市價}")

In [48]:
GetPriceType("LM12T")

Exception: Not correct StockPriceType: LM12T, only can input {LMT: 限價, MKT: 市價, MKP: 範圍市價}

In [17]:
#取得委託單時效性
def __getTimeInForce(timeInForce):
    if timeInForce == "ROD":
        return sj.constant.TFTOrderType.ROD
    elif timeInForce == "IOC":
        return sj.constant.TFTOrderType.IOC
    elif timeInForce == "FOK":
        return sj.constant.TFTOrderType.FOK
    else:
        raise Exception("Not correct TimeInForce: " + timeInForce +
                        ", only can input {ROD, IOC, FOK}")

In [8]:
GetTimeInForce("FOK")

<TFTOrderType.FOK: 'FOK'>

In [18]:
#取得委託單盤別
def __getSessionType(sessionType):
    if sessionType == "RegularTrading":
        return sj.constant.TFTStockOrderLot.Common
    elif sessionType == "AfterHoursTrading":
        return sj.constant.TFTStockOrderLot.Fixing
    elif sessionType == "OddLot":
        return sj.constant.TFTStockOrderLot.Odd
    elif sessionType == "OddLotAtSession":
        return sj.constant.TFTStockOrderLot.IntradayOdd
    else:
        raise Exception("Not correct SessionType: " + sessionType +
                        ", only can input {RegularTrading: 整股, " +
                                          "AfterHoursTrading: 定盤, " +
                                          "OddLot: 盤後零股, " + 
                                          "OddLotAtSession: 盤中零股}")

In [19]:
#取得帳戶資訊
def __getAccountInfo(accountType):
    if accountType == "Stk":
        return api.stock_account
    elif accountType == "Fut":
        return api.futopt_account
    else:
        raise Exception("Not correct AccountType: " + accountType +
                        "only can input {Stk: 取得API預設現貨帳戶資料, Fut: 取得API預設期貨帳戶資料}")

In [20]:
#依帳戶類別取得商品合約，帳戶有股票和期貨，以此為依據去取得Shioaji資料庫中Contracts.Stocks或Contracts.Futures的合約資料結構
def __getContract(symbolId, accountType):
    if accountType == "Stk":
        return api.Contracts.Stocks[symbolId]
    elif accountType == "Fut":
        return api.Contracts.Futures[symbolId]
    else:
        raise Exception("Not correct AccountType:" + accountType + ", and then cause cannot catch contract")

In [30]:
#登出
def Logout():
    api.logout()